In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

df_news = pd.read_csv(r"C:\Users\user\Desktop\python10\week 1 data\raw_analyst_ratings.csv\raw_analyst_ratings.csv")
df_price = pd.read_csv(r"C:\Users\user\Desktop\python10\week 1 data\yfinance_data\yfinance_data\AAPL_historical_data.csv")


Date Alignment

In [8]:
df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce')
df_price['Date'] = pd.to_datetime(df_price['Date'], errors='coerce')

df_news['news_date'] = df_news['date'].dt.date
df_price['price_date'] = df_price['Date'].dt.date

# Merge on the matching date columns
merged = pd.merge(df_news, df_price,
                  left_on='news_date',
                  right_on='price_date',
                  how='inner')

print(merged[['news_date', 'headline', 'stock', 'Close']].head())


    news_date                                           headline stock  \
0  2020-06-05            Stocks That Hit 52-Week Highs On Friday     A   
1  2020-06-03         Stocks That Hit 52-Week Highs On Wednesday     A   
2  2020-05-26                      71 Biggest Movers From Friday     A   
3  2020-05-22       46 Stocks Moving In Friday's Mid-Day Session     A   
4  2020-05-22  B of A Securities Maintains Neutral on Agilent...     A   

       Close  
0  82.875000  
1  81.279999  
2  79.182503  
3  79.722504  
4  79.722504  


Sentiment Analysis

In [13]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.1:
        label = 'Positive'
    elif polarity < -0.1:
        label = 'Negative'
    else:
        label = 'Neutral'
    return pd.Series([polarity, label])

df_news[['sentiment_score', 'sentiment_label']] = df_news['headline'].apply(get_sentiment)

print(df_news[['headline', 'sentiment_score', 'sentiment_label']].head())


                                            headline  sentiment_score  \
0            Stocks That Hit 52-Week Highs On Friday              0.0   
1         Stocks That Hit 52-Week Highs On Wednesday              0.0   
2                      71 Biggest Movers From Friday              0.0   
3       46 Stocks Moving In Friday's Mid-Day Session              0.0   
4  B of A Securities Maintains Neutral on Agilent...              0.0   

  sentiment_label  
0         Neutral  
1         Neutral  
2         Neutral  
3         Neutral  
4         Neutral  


Analysis

In [11]:
#Calculate Daily Stock Returns

df_price['Date'] = pd.to_datetime(df_price['Date'])
df_price = df_price.sort_values('Date')

# Calculate daily return as % change of 'Close' price
df_price['daily_return'] = df_price['Close'].pct_change() * 100  # in percent

# Preview the result
print(df_price[['Date', 'Close', 'daily_return']].head())


        Date     Close  daily_return
0 1980-12-12  0.128348           NaN
1 1980-12-15  0.121652     -5.217061
2 1980-12-16  0.112723     -7.339788
3 1980-12-17  0.115513      2.475091
4 1980-12-18  0.118862      2.899246


In [14]:
# Correlation 

merged_df = pd.merge(df_news, df_price, left_on='news_date', right_on='price_date', how='inner')
# Pearson correlation
correlation = merged_df[['sentiment_score', 'daily_return']].corr(method='pearson')

print("📉 Pearson Correlation Matrix:")
print(correlation)


📉 Pearson Correlation Matrix:
                 sentiment_score  daily_return
sentiment_score         1.000000      0.025741
daily_return            0.025741      1.000000
